# 基于InsightFace的人脸识别
![基于InsightFace的人脸识别](img/1.jpg "基于InsightFace的人脸识别")

## 一、综述  

这篇教程我们将介绍用InsightFace进行人脸检测的基本原理和流程。  
[InsightFace](https://github.com/deepinsight/insightface/tree/master/RetinaFace)是目前深度学习领域SOTA（最先进的，State Of The Art）的人脸分析项目，其中人脸检测（和校正）由RetinaFace实现；而人脸识别由ArcFace实现。  

工业级的人脸识别需要包含4个步骤：  
   * 人脸检测（detection），将图片中的人脸检测切割出来，并计算关键点（landmark）用于人脸矫正。  
   * 人脸矫正（alignment），检测到的人脸可能有各种角度、表情，矫正可以将人脸调整到接近标准模板，提高识别率。  
   * 姿态过滤，利用姿态预估模型计算人脸的姿态，过滤掉偏转角度过大的人脸。  
   * 人脸识别，计算矫正过的人脸的特征码（embedding），计算它与底库特征码的距离，当最小距离小于阀值时，认为识别成功，数据底库特征码对应的身份ID。  
   
![人脸识别的流程](img/3.png "人脸识别的流程")
   
## 二、检测  

人脸检测是把人脸从图片中用矩形框分割出来，这里调用RetinaNet进行处理。  
首先是导入所需的Python库，准备好数据，并定义一些Helper函数。  

In [ ]:
import os
import sys
import random
import datetime
from multiprocessing import  Process

import cv2
import numpy as np
from skimage import transform
import matplotlib.pyplot as plt
%matplotlib inline 

sys.path.append(os.path.join(os.path.abspath(''), 'RetinaFace'))
from retinaface import RetinaFace

In [ ]:
input_picure_path = os.path.join(os.path.abspath(''), 'data/input')

In [ ]:
def get_img_scale(img_shape, scales):
    target_size, max_size = scales
    
    img_size_min = np.min(img_shape[0:2])
    img_size_max = np.max(img_shape[0:2])
    
    img_scale = float(target_size) / float(img_size_min)
    
    if np.round(img_scale * img_size_max) > max_size:
        img_scale = float(max_size) / float(img_size_max)
        
    return img_scale

def draw_on_img(img, faces, landmarks=None):
    for i in range(faces.shape[0]):
        box = faces[i].astype(np.int)
        
        color = (0, 0, 255)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
        
        if landmarks is not None:
            landmark5 = landmarks[i].astype(np.int)
            
            for l in range(landmark5.shape[0]):
                color = (0, 255, 0)
                cv2.circle(img, (landmark5[l][0], landmark5[l][1]), 1, color, 2)
                
    plt.imshow(img[..., ::-1])
    plt.show()

接下来需要设置参数并初始化检测模型。  
要注意当gpu_id小于0的时候，MxNet会自动选用CPU进行计算。  
另外RetinaFace最后一个参数用于选择[Anchor](https://www.quora.com/What-does-an-anchor-mean-in-object-detection)的大小，会影响不同尺寸人脸检测的精度。  

In [ ]:
# consts
score_threshold = 0.94
image_scales = [720, 1024]

# init detector - mxnet selects cpu if gpu_id < 0
gpu_id = -1

detector = RetinaFace('../../share/data/model/R50', 0, gpu_id, 'net3')

最后读出图片并检测就可以了：  

In [ ]:
# list input pics
entries = os.listdir(input_picure_path)

for entry in entries:
    arr = entry.split('.')

    if len(arr) > 1 and arr[1] == 'jpg':
        # read image
        img_path = os.path.join(input_picure_path, entry)
        img = cv2.imread(img_path)

        img_scale = get_img_scale(img.shape, image_scales)
        
        start_time = datetime.datetime.now()
        
        faces, landmarks = detector.detect(
            img,
            score_threshold,
            scales=[img_scale],
            do_flip=False
        )
        
        end_time = datetime.datetime.now()
        time_diff = end_time - start_time
        
        print('{}: spent {} sec(s), detected {} face(s)'.format(
            entry,
            time_diff.total_seconds(),
            faces.shape[0]
        ))
        
        draw_on_img(img, faces, landmarks=landmarks)

### 课堂小实验  
在人脸检测中，我们得到了人脸框和五个关键点，它们的数据格式是怎样的呢？请在下方代码框内，编写程序打印输出它们，并给出你的解释：  

# 三、矫正  

检测到的人脸有各种姿态和角度，如果根据Landmark（关键点）加以矫正，可以提高识别正确率。  
矫正算法的思路是，定义一个标准关键点模板，计算检测到的关键点到标准模板的变换，并对人脸图像执行相同的变换，则实现了对人脸图像的矫正。  
如果图片中有多张人脸，我们只识别最大的一张人脸，这在人脸打卡、闸机等系统是常见处理。对于不同的应用，可以采用不同的过滤规则。  

In [ ]:
def wrap_face(img, faces, landmarks):
    max_size = 0
    max_box= []
    max_landmark = []

    if len(faces) != 0 or len(landmarks) != 0:
        # find out the max face
        for i in range(faces.shape[0]):
            box = faces[i]
            size = (box[0] - box[2]) * (box[1] - box[3])
            
            if size > max_size:
                max_box = box
                max_landmark  = landmarks[i].astype(np.int)
                max_size = size
                
        # config and template
        image_size = [112, 112]
        src = np.array([
            [30.2946, 51.6963],
            [65.5318, 51.5014],
            [48.0252, 71.7366],
            [33.5493, 92.3655],
            [62.7299, 92.2041]], dtype=np.float32)
        
        # template was created for 96x112 pic
        # adjust point position for 112x112
        if image_size[1] == 112:
            src[:, 0] += 8.0
            
        # OK, transform
        warped = None

        if max_landmark is not None:
            tform = transform.SimilarityTransform()
            tform.estimate(max_landmark, src)
            M = tform.params[0:2, :]
            warped = cv2.warpAffine(img, M, (image_size[1], image_size[0]), borderValue=0.0)
            
            plt.imshow(warped[..., ::-1])
            plt.show()
            
            return warped

warped_faces = []

for idx, (img, faces, landmarks) in enumerate(zip(img_list, bbox_list, landmark_list)):
    warped_face = wrap_face(img, faces, landmarks)
    
    warped_faces.append(warped_face)

## 四、识别

人脸识别问题的最大的困难在于，相对于普通的分类任务，人脸(作为类别)是数量不定并且非常多的。  
为了解决这个问题，通常人脸识别模型本身就是分类模型，但是训练方法上与普通分类有很大区别。  

我们人类进行人脸识别的时候，可以在内心衡量两张脸之间的相似度，同一张脸的不同图片，它们是比较相似的；而不同脸的图片则差别较大。  
同样的道理，我们也可以用神经网络来计算图片的特征向量，并且比较特征之间的相似度。相似度的计算，可以用数学公式表示成距离函数。  

因此，人脸识别模型是利用Metric Learning的方法进行训练的。比如Triplet Loss，训练的时候，网络会趋向于拉近来自同一张脸的图片间的距离、同时推远来自不同的脸的图片间的距离。  
显然，欧式距离并不是唯一可用的距离函数，ArcFace就提出了更好的人脸特征距离。  

ArcFace是一个角度距离，如果我们把两个图片的特征向量进行归一化，那么它们之间的距离，就等于特征向量的夹角。  
而归一化向量的点积就等于向量夹角的余弦，所以我们对向量点积取一个反余弦就可以得到夹角弧度，这就是ArcFace名字的由来。  
那么为什么我们要把欧式距离变成角度距离呢？这是因为我们并不关心同一张脸不同图片的区别，所以我们把同一张脸的所有图片都映射到一个角度就足够了。  
![ArcFace](img/2.png "ArcFace")

好了，理论知识已经足够了，让我们继续一些实际的工作，首先需要建立特征底库。  

In [ ]:
class ExtractFeature:
    def __init__(self, prefix, epoch, gpu_id=-1):
        sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
        
        if gpu_id < 0:
            self.mod = mx.mod.Module(symbol=sym, context=mx.cpu(0))
        else:
            self.mod = mx.mod.Module(symbol=sym, context=mx.gpu(gpu_id))

        # 获取网络结构
        internals = self.mod.symbol.get_internals()

        # 获取从输入到特征值层的模型
        feature_net = internals["fc1_output"]

        if gpu_id < 0:
            self.feature_model = mx.module.Module(symbol=feature_net, context=mx.cpu(0))
        else:
            self.feature_model = mx.module.Module(symbol=feature_net, context=mx.gpu(gpu_id))

        data_shapes = [('data', (1, 3, 112, 112))]
        self.feature_model.bind(data_shapes=data_shapes)

        # 上面只是定义了结构，这里设置特征模型的参数为Inception_BN的
        self.feature_model.set_params(
            arg_params=arg_params,
            aux_params=aux_params,
            allow_missing=False
        )

    def bufferDecode(self, str_image):
        data = np.fromstring(str_image, dtype='uint8')
        return cv2.imdecode(data, 1)

    def do_flip(self, data):
        for idx in xrange(data.shape[0]):
            data[idx, :, :] = np.fliplr(data[idx, :, :])

    def ExtractFeature(self, image, b_flip):
        if isinstance(image, str):
            imgsrc = cv2.imread(image)
        else:
            imgsrc = image
            
        img = cv2.resize(imgsrc, (112, 112), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img = np.swapaxes(img, 0, 2)
        img = np.swapaxes(img, 1, 2)

        embedding = None

        fliplist = []
        
        if b_flip == True:
            fliplist = [0, 1]
        else:
            fliplist = [0]

        for flipid in fliplist:
            _img = np.copy(img)
            if flipid == 1:
                self.do_flip(_img)

            Batch = namedtuple('Batch', ['data'])
            data = Batch([mx.nd.array(np.array([_img]))])

            # 输出特征
            self.feature_model.forward(data, is_train=False)
            _embedding = self.feature_model.get_outputs()[0].asnumpy()
            if embedding is None:
                embedding = _embedding
            else:
                embedding += _embedding

        _norm = np.linalg.norm(embedding)
        embedding /= _norm

        return embedding

In [ ]:
# consts
recog_model_prefix = '../../share/data/model/feature'
model_epoch = 0

# mxnet selects cpu if gpu_id < 0
extractor = ExtractFeature(recog_model_prefix, model_epoch, gpu_id=-1)

In [ ]:
# list db pics
entries = os.listdir(db_picture_path)

id_list = {}

for entry in entries:
    arr = entry.split('.')
    
    if len(arr) > 1 and arr[1] == 'jpg':
        img, bboxes, landmarks = detect(db_picture_path, entry)
        wraped_face = wrap_face(img, bboxes, landmarks)
        
        # get embedding
        embedding = extractor.ExtractFeature(wraped_face, False)
        
        # save embedding
        id_list[arr[0]] = embedding

### 课堂小实验  
请试着计算底库中任意2个特征向量的欧式距离吧！（提示，用[numpy.linalg.norm](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html)）  

终于，万事俱备，只欠——特征比对：

In [ ]:
# 相似度计算函数
def calc_similarity(Feature1, Feature2):
    return np.dot(Feature1, Feature2) / (np.linalg.norm(Feature1) * np.linalg.norm(Feature2))

In [ ]:
for i, face in enumerate(warped_faces):
    face_feature = extractor.ExtractFeature(face, False)

    face_scores = []

    for name in id_list.keys():
        id_feature = id_list[name]

        similarity = calc_similarity(face_feature.squeeze(), id_feature.squeeze())
        face_scores.append(similarity)

    max_score = np.max(face_scores)
    max_id = np.argmax(face_scores)

    if max_score > 0.00:
        name = id_list.keys()[max_id]
    else:
        name = 'Unknown'

    print('pic: {}, id: {}, similarity: {}'.format(i, name, max_score))
    plt.imshow(face[..., ::-1])
    plt.show()

## 五、课后作业  

利用人脸相似度算法，我们可以构建一个“名人鉴定机”，输入一张你的图片，输出和你长得最像的名人的脸，快去收集名人照片，用我们学到的知识做起来吧！  